# Experiment 4: Latest features on all statistical classifiers

In [1]:
#######################
##### Duplicated! #####
#######################

from protos import review_set_pb2, review_pb2
review_set = review_set_pb2.ReviewSet()
with open("data/yelpZip", 'rb') as f:
  review_set.ParseFromString(f.read())

In [2]:
#######################
##### Duplicated! #####
#######################

from sklearn.utils import shuffle

fake_reviews = list(filter(lambda x: x.label, review_set.reviews))
count_fake = len(fake_reviews)
genuine_reviews = []
unused_genuine_reviews = []
counter_genuine = 0
for review in shuffle(review_set.reviews):
  if review.label == True:
    continue
  if counter_genuine <= count_fake:
    genuine_reviews.append(review)
    counter_genuine += 1
  else:
    unused_genuine_reviews.append(review)
  
concatted_reviews = shuffle(fake_reviews + genuine_reviews)
print("fake:", len(fake_reviews))
print("real:", len(genuine_reviews))
print("all:", len(concatted_reviews))
print("unused real:", len(unused_genuine_reviews))

fake: 80466
real: 80467
all: 160933
unused real: 447665


In [3]:
#######################
##### Duplicated! #####
#######################
def hi():
  """
from exp2_feature_extraction import find_words
from sklearn.utils import shuffle

num_each_class = 8141
reviews = shuffle(review_set.reviews)

i = 0
fake_reviews = []
for x in reviews:
  if i == num_each_class:
    break
  if x.label:
    fake_reviews.append(x)
    i+=1

i = 0
genuine_reviews = []
for x in reviews:
  if i == num_each_class:
    break
  if x.label == False:
    fake_reviews.append(x)
    i+=1
    
all_reviews = shuffle(fake_reviews + genuine_reviews)
  """

In [3]:
targets = [x.label for x in concatted_reviews]

In [4]:
from latest_feature_extraction import get_features_maker

## Linear Discriminant Analysis

In [58]:
get_features = get_features_maker(concatted_reviews, 775)

In [59]:
predictor_features = get_features(concatted_reviews)

Here we can only make our bag of words features so big before we crash with memr

In [60]:
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.model_selection import cross_validate
import numpy
linearDA = LinearDiscriminantAnalysis()
results = cross_validate(linearDA, predictor_features.toarray(), numpy.asarray(targets), cv=2,
                         return_train_score=False)

/home/stefan/anaconda3/envs/lucas/lib/python3.7/site-packages/sklearn/discriminant_analysis.py:388: UserWarning: Variables are collinear.
  warnings.warn("Variables are collinear.")
/home/stefan/anaconda3/envs/lucas/lib/python3.7/site-packages/sklearn/discriminant_analysis.py:388: UserWarning: Variables are collinear.
  warnings.warn("Variables are collinear.")


In [61]:
sum(x for x in results['test_score'])/2

0.6899020151921419

## Feature scaling

We normalise all our features to be between one and zero. We need to do this to suppress the mega features vs tiny features situation. Most classifiers use Euclidian distance, which has no knowledge of the units being used.

In [21]:
n = 1000

In [22]:
predictor_features2 = get_features_maker(concatted_reviews[:n])(concatted_reviews[:n])

Our sparse features are problematic here because we need to conver our data to array for standardisation. We have to limit the number of features because of this

In [23]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler().fit(predictor_features2.toarray())
scaler

StandardScaler(copy=True, with_mean=True, with_std=True)

In [25]:
scaled_features = scaler.transform(predictor_features2.toarray())

## Support Vector Machine

In [6]:
from sklearn.svm import SVC

svc = SVC(kernel="sigmoid", gamma='scale', random_state=42)

In [ ]:
svc = SVC(kernel="sigmoid", gamma='scale', random_state=42)

In [8]:
# Should already be imported
from sklearn.model_selection import cross_validate

In [26]:
cross_validate(svc, scaled_features, targets[:n], cv=2, return_train_score=False)

{'fit_time': array([3.42629886, 2.06137514]),
 'score_time': array([1.97278214, 1.91485572]),
 'test_score': array([0.55688623, 0.55711423])}